In [ ]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=a4055e2c61ea1d97b84822d1df11b9e7ca5fb2c19811f3bc86c9661af3777b94
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.1 MB/s eta 0:00:00


In [ ]:
!pip install open_clip_torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 9.4 MB/s eta 0:00:00


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import timm

In [ ]:
import wandb
import torchvision as tv
import fastai
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
import open_clip


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip -qq '/content/drive/My Drive/Geolocation/50States10K.zip' -d '/content/geolocation'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
config = SimpleNamespace(
    batch_size=32,
    img_size=224,
    seed=42,
    pretrained=False,
    normalize=True,
    model_name="vit_small_patch32_224",
    epochs=10,                                 # FIX THIS LATER
    learning_rate=2e-3,
    resize="crop",
)

In [ ]:
def load_data():
  resize_method = ResizeMethod.Squish if config.resize == "squish" else ResizeMethod.Crop
  dls = fastai.vision.data.ImageDataLoaders.from_folder(
    '/content/geolocation',
    valid_pct=0.2,
    seed=config.seed,
    bs=config.batch_size,
    pretrained=False,
    item_tfms=Resize(config.img_size, method = resize_method),
  )


  mean, std = (0.48145466, 0.4578275, 0.40821073), (
      0.26862954,
      0.26130258,
      0.27577711,
  )

  if config.normalize:
      dls.add_tfms([Normalize.from_stats(mean, std)], "after_batch")

  return dls

dls = load_data()
print("Num classes =", dls.c)
print(len(dls.train_ds), len(dls.valid_ds))


Num classes = 50
400022 100005


In [ ]:
# callback = WandbCallback(log_preds=True, log_preds_every_epoch=True, log_model=True)
from fastai.callback.wandb import log_model
def new_after_epoch(self):
  "Log validation loss and custom metrics & log prediction samples"
  # Correct any epoch rounding error and overwrite value
  self._wandb_epoch = round(self._wandb_epoch)
  if self.log_preds and self.log_preds_every_epoch:
      self.log_predictions()
  wandb.log({'epoch': self._wandb_epoch}, step=self._wandb_step)
  wandb.log({n:s for n,s in zip(self.recorder.metric_names, self.recorder.log) if n not in ['train_loss', 'epoch', 'time']}, step=self._wandb_step)


  name = f"{learn.cbs[-2].fname}_epoch_{self._wandb_epoch}"
  learn.save(name, with_opt=True)
  # metadata = {n:s for n,s in zip(self.recorder.metric_names, self.recorder.log) if n not in ['train_loss', 'epoch', 'time']}
  # log_model(self.save_model.last_saved_path, name=self.save_model.fname, metadata=metadata) 
  # log_model(f"{name}.pth", name=self.save_model.fname+self._wandb_epoch, metadata=metadata)     



WandbCallback.after_epoch = new_after_epoch

In [ ]:
train_cbs = [
    WandbCallback(log_preds=True, log_preds_every_epoch=True, log_model=True),
]

other_cbs = [
]

top_2_accuracy = functools.partial(top_k_accuracy, k=2)
top_3_accuracy = functools.partial(top_k_accuracy, k=3)
top_5_accuracy = functools.partial(top_k_accuracy, k=5)
top_10_accuracy = functools.partial(top_k_accuracy, k=10)

learn = vision_learner(
    dls,
    config.model_name,
    metrics=[accuracy, top_2_accuracy, top_3_accuracy, top_5_accuracy, top_10_accuracy],
    # metrics=[accuracy, top_2_accuracy, top_3_accuracy],
    concat_pool=True,
    cbs=train_cbs
).to_fp16()

learn.model.to(device)
None

In [ ]:
# learn.cbs[-2].fname

In [ ]:
def wandb_init():
    wandb.login()

wandb_init()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
"""path='/content/geolocation/models/model.pth'
path = Path(path)
if not path.is_file():
    raise f'path must be a valid file: {path}'
name = None
name = ifnone(name, f'run-{wandb.run.id}-model')
_format_metadata(metadata)    
artifact_model = wandb.Artifact(name=name, type='model', metadata=metadata, description=description)
with artifact_model.new_file(str(Path(name).with_suffix(".pth")), mode='wb') as fa:
    fa.write(path.read_bytes())
wandb.run.log_artifact(artifact_model)
"""

'path=\'/content/geolocation/models/model.pth\'\npath = Path(path)\nif not path.is_file():\n    raise f\'path must be a valid file: {path}\'\nname = None\nname = ifnone(name, f\'run-{wandb.run.id}-model\')\n_format_metadata(metadata)    \nartifact_model = wandb.Artifact(name=name, type=\'model\', metadata=metadata, description=description)\nwith artifact_model.new_file(str(Path(name).with_suffix(".pth")), mode=\'wb\') as fa:\n    fa.write(path.read_bytes())\nwandb.run.log_artifact(artifact_model)\n'

In [ ]:
with wandb.init(config=config, project="geolocation"):
    # log_model('model.pth')
    print(f"Finding optimal learning rate")
    # lrs = learn.lr_find()
    # lr = 0.002511886414140463
    # lr = lrs.valley

    print(f"Creating learner for {config.model_name}")
    # print(f"Learning rate = {lr}")
    ti = time.perf_counter()
    # learn.fine_tune(base_lr=lr, epochs=config.epochs)
    learn.unfreeze()
    learn.fit_one_cycle(config.epochs)
    wandb.summary["fit_time"] = time.perf_counter() - ti

wandb: Currently logged in as: ben_z. Use `wandb login --relogin` to force relogin


Finding optimal learning rate
Creating learner for vit_small_patch32_224


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,top_k_accuracy,top_k_accuracy,top_k_accuracy,time
0,3.053133,2.960406,0.207000,0.318764,0.396690,0.512824,0.693245,15:05
1,2.987851,2.868047,0.228019,0.341753,0.421189,0.537923,0.711854,15:05
2,2.708107,2.628685,0.281666,0.406230,0.491065,0.605050,0.767582,15:21
3,2.533586,2.426412,0.328944,0.464957,0.549743,0.661057,0.809080,15:17
4,2.340391,2.242872,0.373611,0.512554,0.597430,0.704865,0.844558,15:24
5,2.064002,2.089016,0.413729,0.553082,0.636168,0.739713,0.865557,15:26
6,1.725158,1.962427,0.448468,0.587691,0.668367,0.765402,0.881056,15:26
7,1.443653,1.911446,0.473056,0.610160,0.687806,0.781241,0.888626,15:27
8,1.157728,2.006458,0.477566,0.612239,0.688946,0.780411,0.886296,15:23
9,0.832030,2.155201,0.471146,0.603760,0.679706,0.770881,0.878506,15:39


Better model found at epoch 0 with valid_loss value: 2.9604063034057617.
Better model found at epoch 1 with valid_loss value: 2.868046522140503.
Better model found at epoch 2 with valid_loss value: 2.628685474395752.
Better model found at epoch 3 with valid_loss value: 2.426412343978882.
Better model found at epoch 4 with valid_loss value: 2.2428722381591797.
Better model found at epoch 5 with valid_loss value: 2.0890159606933594.
Better model found at epoch 6 with valid_loss value: 1.9624265432357788.
Better model found at epoch 7 with valid_loss value: 1.9114460945129395.


accuracy,▁▂▃▄▅▆▇███
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
mom_0,██▇▆▅▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████
mom_1,██▇▆▅▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████
raw_loss,█▇▆▆▆▆▅▆▆▅▅▆▆▄▅▅▄▄▄▄▅▃▃▄▃▂▃▄▃▃▂▃▂▃▂▁▁▁▂▂
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sqr_mom_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [ ]:
wandb.init()

In [ ]:
learn.show_results()

KeyboardInterrupt: ignored

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
learn.dls.c

In [ ]:
interp.plot_top_losses(10)

In [ ]:
interp.plot_confusion_matrix(figsize=(15,15), dpi=60)